References:
- The order of logits [contradiction, neutral, entailment]: https://joeddav.github.io/blog/2020/05/29/ZSL.html
- the Seq2SeqSequenceClassifierOutput class: https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_outputs.Seq2SeqSequenceClassifierOutput 

In [1]:
import pandas as pd
from transformers import pipeline
from tqdm.notebook import tqdm
import numpy as np
tqdm.pandas()
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [2]:
# pipeline for quick inference
loaded_classifier_2 = pipeline("zero-shot-classification", model='../models/pretrained/bart-large-mnli/')
classifiers = {
    'BART': loaded_classifier_2,
}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# using the model and tokenizer separately to explore in-depth
nli_model = AutoModelForSequenceClassification.from_pretrained('../models/pretrained/bart-large-mnli/')
tokenizer = AutoTokenizer.from_pretrained('../models/pretrained/bart-large-mnli/')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
candidate_labels = ['very recent breaking news on major railway transportation disruption and closure, bad news',
'later reports of past transportation disruption event, bad news',
'lawsuits, legal or insurance impact of past event, bad news',
'general social, business, economic reports, studies and trends',
'leisure news',
# 'other non-critical news',
]

In [5]:
test_candidate_labels = ['lawsuits of past event',
                    'other news']

In [6]:
articles = [""" 
Since returning to work following the derailment of a Norfolk Southern train carrying hazardous chemicals, Rick Feezle said he has had a raspy voice and chest pain.
His wife has experienced sore throats and headaches, he said.
Feezle, 63, said he has worked in the area around East Palestine, Ohio, all of his adult life and operates two businesses there: a salvage yard and an auto shop.
He's part of a group of people who live or work near the derailment site who have filed a class action suit against Norfolk Southern. The Feb. 3 incident resulted in a fire and chemical spill, forcing residents within a roughly 1-mile radius to evacuate. Several days later, the rail company released and burned vinyl chloride — a flammable gas — a move officials said would alleviate the risk of an explosion.

Authorities said residents could return home two days after that.

"Nobody can tell us what we should do other than 'It’s safe, go head on back in there,'" Feezle said, his voice crackly. "And the fish are dying and animals are dying and I can hardly talk and my chest hurts."
His lawsuit is one of at least six class action suits already filed against Norfolk Southern since the accident. For the most part, those suing the company allege that they’ve lost income due to evacuations, were exposed to cancer-causing chemicals and no longer feel safe in their homes.

Norfolk Southern said it was "unable to comment directly on litigation." But in a public update on Thursday, the company noted that in addition to its ongoing cleanup work, it was distributing more than $2 million in financial assistance to affected families and businesses to help with the costs of the evacuation, as well as creating a $1 million fund for the community.
Feezle and his fellow plaintiffs are requesting compensation from the company for lost business revenue and expenses incurred during their evacuations. They’re also seeking punitive damages for exposure to toxic chemicals.

The Environmental Protection Agency classifies vinyl chloride as a carcinogen, and routine exposure can increase one’s risk of liver cancer or damage.
Another recently filed class-action, not involving Feezle, alleges that the rail company "discharged more cancer causing Vinyl Chloride into the environment in the course of a week than all industrial emitters combined did in the course of a year" in the U.S.

Lisa Sodergen, a plaintiff in that suit, said in legal filings that her house was “surrounded by toxic black smoke” that irritated her lungs, eyes and skin. Sodergen lives in Lawrence County, Pennsylvania, about 5 miles from the derailment site, which is outside the evacuation zone.
“She lives with ongoing pulmonary irritation and fear for the long-term consequences to her health and water supply,” the suit alleges.

Along with another plaintiff, she is seeking damages for the increased risk of future illness and costs of medical monitoring to ensure the early detection of disease.

Rene Rocha, an attorney at Morgan & Morgan Lawyers who represents Sodergen, said both plaintiffs in that case have had ongoing health issues that arose after the derailment.

"When you talk to them, they run out of breath or start coughing," he said.

As of Thursday, the EPA has helped evaluate the indoor air in more than 500 homes, in conjunction with Norfolk Southern, and has not detected vinyl chloride above levels of concern in any of them. Gov. Mike DeWine said Thursday that the municipal water was safe to consume, based on the results of sampling and tests done by the EPA, Norfolk Southern and other agencies.

In an open letter, Norfolk Southern CEO Alan Shaw promised to stay in the area “as long as it takes to ensure your safety and to help East Palestine recover and thrive.”
Feezle said his employees — which include many of his family members — are still drinking bottled water, since they’re not convinced that the water supply is free of contamination. The Ohio Department of Health is recommending that people who get their drinking water from private wells use bottled water until those wells are tested.
Feezle also said he has heard customers complaining about the lingering stench in the air. The EPA said last week that byproducts of vinyl chloride can emit smells at levels lower than what is considered hazardous.

Before making decisions about whether to keep operating his businesses in East Palestine, Feezle said, he’s waiting on independent water and air quality tests, which are being organized by his lawyers starting next week. Until the derailment, he said, he had been planning to pass his two businesses on to his children and employees one day.

“If it’s bad and it’s gonna kill us, let us leave,” he said. “It’s gonna be a lot of bankrupt people and a lot of people that will take a long time to get started again, but least we’ll be alive.”

Feezle also owns several rental properties in town, at least four of which are in the evacuation zone, he said.

“The first person that really gets sick, that town is going to be a ghost town,” he said. “My businesses will be closing down, and I’ve worked all my life. I started in business when I was 19 and worked for myself that whole time. And my property values are going to plummet. I believe it cost me millions of dollars, the train wreck.”


On top of all this, Feezle added, he is scared that he or his family members might develop cancer someday.

“I’m afraid we’re all gonna be out of here in five years, one way or another," he said

""",
"""
The costs associated with Norfolk Southern's fiery February derailment in Ohio have more than doubled to $803 million as the railroad works to clean up the mess and moves forward with all the related lawsuits.

Norfolk Southern recorded another $416 million charge related to the East Palestine derailment on Thursday as part of its second-quarter earnings after previously announcing a $387 million charge earlier this year. Most of the costs are related to the cleanup of the hazardous chemicals that were released, but $222 million is a combination of legal fees and the $63 million of assistance it has offered to the community. The company faces a number of class-action lawsuits as well as a suit filed by Ohio authorities and a federal civil suit brought by the Justice Department and the Environmental Protection Agency. 

The derailment near the Ohio-Pennsylvania border prompted a national conversation about railroad safety after thousands of people had to evacuate when officials decided to blow open several tank cars filled with vinyl chloride, a gas used to make plastic, because they believed they might explode. The resulting fire sent a towering plume of black smoke over the town three days after the derailment spilled several other hazardous chemicals, including butyl acrylate, ethylene glycol monobutyl ether, ethylhexyl acrylate and isobutylene. The company said in February that the derailment contaminated at least 15,000 pounds of soil and 1.1 million gallons of water.

The $803 million cost estimate doesn't include funds to compensate the East Palestine community for any long-term health effects, drop in home values or drinking water issues because those are still being negotiated, so the total will grow. Since the derailment, residents have expressed fears about drinking tap water, even though state officials say municipal drinking water is safe to consume. But Norfolk Southern also expects to eventually recover some of those costs from its insurance and lawsuits against other companies involved in the derailment.

The additional charges related to the derailment, combined with a 6% drop in the number of shipments the railroad delivered, more than halved the Atlanta-based company's profit to $356 million, or $1.56 per share. That's down from $819 million, or $3.45 per share, a year ago.
""",
""" 
About 20 cars of a Norfolk Southern cargo train derailed near Springfield on Saturday evening, the second derailment of the company’s trains in Ohio in a month, officials said.

But unlike the 3 February derailment in East Palestine, Ohio, a company spokesperson said there were no hazardous materials aboard the train, the Columbus Dispatch reported.
The East Palestine accident caused a major environmental problem and has stoked outrage across the US at the train company and a government response perceived by many residents as being lackluster.

The latest train, which did not have passengers, derailed around 5pm on Saturday by state route 41, near the Clark county fairgrounds, the Dispatch reported. Springfield is about 46 miles (74km) west of the state capital of Columbus, Ohio.
The 20 cars of the 212-car train derailed while traveling south, the Norfolk Southern spokesperson said.

Shawn Heaton told the Springfield News-Sun that he was waiting at an intersection as the train crossed and captured the start of the derailment on video.

“I was right there and I was playing on my phone and then I heard a loud bang. And when I heard the loud bang, I started recording,” Heaton said. “When I heard the bang, there was all kinds of debris and metal shoot out from under the cars and that’s when I started recording and you could see them start jumping off the tracks.”

The Clark county emergency management agency has asked residents within 1,000ft of the derailment to shelter in place, but the agency said it has not issued formal evacuation orders.

On 3 February, 38 cars of a Norfolk Southern freight train in East Palestine, in north-east Ohio near Pennsylvania, derailed and several of the train’s cars carrying hazardous materials burned.

Though no one was injured, nearby neighborhoods in both states were imperiled. The crash prompted an evacuation of about half the town’s roughly 5,000 residents, an ongoing multigovernmental emergency response and lingering worries among villagers of long-term health impacts.

""",
""" 
On June 2 , 2023 288 people were killed in a railway crash near Bahanaga Bazaar Railway Station in Odisha, making this India's worst train accident since 1995. The discussion on railway safety this has provoked has largely ignored the working conditions of railway workers, perhaps the largest public-sector workforce in the country. The following two articles consider the working conditions of railway train drivers, known in the Indian Railways as 'loco pilots' (LPs), particularly freight or goods LPs who constitute roughly 70% of the total LPs in the country. The information for these articles was collected through interviews with members of the Loco Running Staff Association (LRSA), a trade union of approximately 44,000 railway running staff. Read the first article here, on LPs' working hours, here. The second article looks at LPs' working conditions, the intensification of work and the poor condition of railway infrastructure and how this results in higher stress levels for LPs. § The intensification of LPs' work is not only a matter of the extension of working hours. The addition of duties in an increasingly poorly maintained railway system makes their work increasingly stressful and dangerous. We can observe this at every step of an LP's shift, which includes two components: a "running time", when the train is being driven, and a period for preparatory and complementary work before and after this. Preparatory work consists of handing over and taking charge of the train, reading the route's caution orders (for instance, where on the route the train should be driven at a slower speed, where faults have been reported on the tracks etc.) and undergoing a breathalyser test. Then the LP must prepare, check and activate the engines; check the breaks through a break-fail test; check the lights and oiling; and check that the logbook of defects has been attended to. And it is here that the problems start. Today, goods trains are longer (with sometimes 68 wagons, at a length of up to 3.5 km), have an increased freight load (from 1,600 tonnes 20 years earlier, to 6,000 tonnes today) and must be run faster (from an earlier speed of 40 km/hr-85 km/hr to 90-100 km/hr or 120-160 km/hr in the case of super-fast trains) from what they were earlier. Clearly, train marshalling, the goods load and maintenance of rolling stock must be paid close attention to, to ensure the safety of the train. Yet, LRSA members claim it is precisely these processes which have been undermined by sub-contracting in recent years. LPs claim that since train marshalling has been sub-contracted, train formation rules are being flaunted, which increases the chances of derailment. For instance, goods trains must have some empty wagons placed between loaded wagons, which can relieve some of the shock of a break on the loaded wagons. However, these empty wagons are not being coupled to trains, leading to the possibility of loaded wagons lifting if the LP has to break at speed, possibly derailing or even capsizing the train. Moreover, LPs claim contractors frequently load goods trains with coal, iron ore etc. beyond the maximum markings on railway wagons, which can lead to axle breakage and the breaking of trains. Moreover, LPs claim that the sub-contracting of the work of the Carriage and Wagon departments in loco sheds has led to compromises in the maintenance of rolling stock. While earlier a train was checked at a Carriage and Wagon depot every 400 km, trains now often run long routes, such as from Mumbai to Kolkata, without a single check. Sometimes trains run more than one trip without being checked. In this context of falling safety standards, much of the work of checking of trains before departure has been put onto the shoulders of LPs and guards - a technical task they are not trained to do. For instance, LPs are required to check the train's axle and the goods load that they are carrying. But neither LPs nor guards have any formal training in this department, and argue they are unable to properly check whether the goods have been loaded safely. And when an LP does notice a defect and draws the attention of the chargeman to it, LPs claim that, especially on night shifts, sometimes there simply aren't maintenance workers to rectify the issues due to staff vacancies. Similarly, when LPs report errors on the track, they are not attended to - in some instances, these are errors which have caused derailments. The slackening of safety standards and procedures due to subcontracting results in increased stress to the LP's heart. Sometimes a signal accidentally bobs when being cleaned, and the train going at 100 km/hour must be suddenly stopped which seriously affects the LP's heart rate. Similarly, while railway procedure requires that two stops at distances of 600 m and beyond that a red-flag man at 45 m should signal to LPs to stop when electric wire maintenance is taking place, private contractors only station one red-flag man. LPs claim that this is insufficient warning to stop a train going at 100 km/hour, and that this has already led to cases of maintenance workers getting hurt. As one exasperated LP put it to me after describing the state of safety in the Railways: "Sab bhagwan-bharose chal raha hai (Everything has been left at god's mercy)." An extreme working environment The second part of the LP's shift, the running time, consists of starting the train, exchanging the green signal with the guard and station master, and piloting the engine. During running, the LP needs to control the speed of the train in line with regulations and the caution orders for the section, constantly check the overhead electrical wire to see that no part of it is hanging and check the air pressure and break power. The LP also needs to watch for animals and human beings trespassing on the track. Apart from this, LRSA officials claim goods LPs are increasingly being forced to take on extra duties - for instance, working multiple trains in one spell of duty and being made to stop at stations and shunt other trains before picking up their journey again. During running, the conditions of the loco cab have made loco piloting a stressful and dangerous job. For there are fundamental problems with the design of the loco cabs where LPs must spend most of their working lives. There is no air temperature regulation within the cab, where temperatures rise to over 60°C in summer and drop to near zero in winter, with wind entering through crevices in doors and windows. There are no toilets in most locomotives, an aspect of railway 'design' that has remained unchanged since the beginning of the Indian railways in the 1850s. There is no proper sound proofing of the cab, where noise levels reach as high as 98.4 decibels, when the Occupational Safety and Health Administration of the United States Department of Labour claims that regular exposure to over 85 decibels for eight hours a day increases the chances of hearing loss. At times of reduced visibility, for instance during the North Indian winter smog and mist, LPs must keep blowing the train horn, which increases the noise in the cab even further. LPs' seats are uncomfortable and too far from the work panel, which forces them to constantly lean over or keep standing through the trip. An article in Frontline claimed that the cushioned seats and air conditioning in imported wagons are deliberately removed before being put to use on the Indian railways since "those at the top" claim that in a comfortable chamber the LP will fall asleep. In a 2010 study conducted by a team of railway doctors quoted in the HPC, of 100 LPs, 96% identified "postural discomfort and non-spacious work place" and 95% "noisy workplace" as leading reasons for build-up of stress. These stresses and strains play out on the LPs body and mind, affecting his/her concentration. "Signal kahin bhi daal dete hai - pahaad pe, jhopdi pe" Working under these conditions, LPs claim that observing and complying with signals is becoming increasingly difficult for them given the increase in the number of signals, the flouting of signal placement norms and the dilapidation of the signalling system. The number of signals on railway routes has dramatically increased. For instance, LPs report that on Mumbai's Chhatrapati Shivaji Maharaj Terminus-Kalyan route, signals, which used to be at distances of two km from each other, are now at a stone's throw from each other. So, in the journey from CST-Kalyan-CST, there are no less than 400 signals which the LP must observe and comply with. To take another example, on the Chennai Central-Vijayawada section of the Southern Railway, over 430 km there are 386 signals, i.e., almost one per kilometre. On this route LPs on trains with speeds of 120-130 km/hour encounter a signal every 27 seconds. While greater signalling is no doubt a positive development, for the overworked and under-slept LP, monitoring hundreds of signals causes continuous mental stress, and can only be effective if their working hours are reduced. The stresses on the LP are increased by the fact that signal placement regulations are increasingly being flouted. Railway rules require that signals be placed uniformly at eye level on the left-hand side of the track it controls, or if there isn't space, then above it. One can imagine that this regularity is essential for an LP to properly observe and comply with signals at speed for running times extending to 11 hours and more, particularly when their required area of observance and concentration keeps increasing with single line sections being converted to double and quadruple lines and the erection of overhead electrical equipment. However, LPs claim that sub-contractors involved in signal construction have flouted rules regarding signal placement, sometimes putting signals on the right side of the track or at an elevation. An LP described the arbitrariness of signal placement in the Mumbai area this way: "Signal kahin bhi daal dete hai - pahaad pe, jhopdi pe (The signal is placed just anywhere - on a mountain, on a hut)." When signal placement is irregular and more than one track is running parallel, it is difficult for LPs to know which signal is for their track, and which for the adjacent line. Moreover, in many places, there are no arrow marks pointing to the signal, which further confuses the LP in a situation which requires quick reactions. These factors greatly increase the possibility of LPs not being able to comply with signals on time, a situation which is known as "Signal Passed At Danger" (SPAD). Indeed, on August 31 on the above mentioned CST-Kalyan line, two local trains nearly collided, with Railway officials claiming that "the incorrect positioning of a signal post" might have led to the incident. This is to say nothing about the fact that in 2017 there was an eight-year backlog on replacing signal gears, creating a situation where many small signal failures take place regularly on the network. Indeed, as V. Sridhar has shown, a third of accidents due to equipment failure in 2021-22 were the result of "failure of signal apparatus", and there are currently 14,850 vacancies in the Signalling Department of the Indian Railways. Yet despite these problems which do not allow them to attain proper speed, LPs claim they are fined and penalised by the railway administration if they arrive late at their destination. This pushes them to compromise on caution. One driver said, "We lose our minds - if I have a yellow signal how can I go faster? But if I don't, I'll be penalised..." The toll it takes: "Retirement ke pehle hi khatam ho jaate hain" On top of all of this, goods LPs must also go through the traumatic experience of recovering and moving corpses from the tracks. When trains are running at high speeds animals and human beings trespassing on the tracks are sometimes unavoidably run over. When this happens, goods LPs must, along with guards, stop the train and clear the tracks of the corpse, which is often cut into pieces - a highly traumatic event. Yet, LPs claim the Indian Railways does not provide them with the necessary safety equipment to do this - not even gloves. The 2010 study by Railway Doctors quoted in the HPC cited earlier found that the various stressors in an LP's work produce "disrupted sexual functions", muscle aches and pains, sleep disturbances, headaches, stomach problems, "irritability and anger", and "frustration and anxiety". Many LPs also speak of blood pressure issues and diabetes being endemic among LPs. Due to the shortage of breaks, their family and social time is non-existent, which could help them deal with stress. One dejected LP replied when I asked about the life expectancy of LPs: "Retirement ke pehle hi khatam ho jaate hain (Death comes before retirement)." Despite all these problems, rail accidents are blamed on the LP. But it stands to reason that the opposite is true. Given these conditions, it is a testament to the abilities of LPs and their concern for safety that more accidents do not happen. It would be more accurate to say that railway workers are in fact keeping the railways safe despite their hours and conditions of work. But these working hours and conditions take their toll. As this article reached completion, CSMT ALP Sujit Jayant reportedly died by suicide at Kalyan on August 14. While the immediate issue was reported harassment by railway officials who were forcibly excluding him from work till September, LRSA officials pointed out that the conditions of an LPs life and work were also a contributory factor. As a crowd of LPs gathered to protest against the harassment which led to Jayant's death, they protested at the use of the word "suicide" to describe it. They insisted: "Murder hua hai (This was a murder)."
""", 

"""
Ukrainian state railway company Ukrzaliznytsia jointly with American billionaire Howard Buffett have created the Food Train, the first ever train-kitchen in the world. It is able to make up to 70,000 meal portions during the week in an autonomous regime without power. Source: social media accounts of Ukrzaliznytsia Quote: "Last winter railway stations became refuge centres in which Ukrainians could get warm and eat in frontline regions amid regular blackouts. This year in addition to railway stations we also created an entire train able to generate over 10,000 food portions a day anywhere and at any time." The Food Train can work autonomously for 5-7 days, and its equipment allows for cooking full meals: soup, porridge, salads, meats etc. Special thermal boxes installed inside the train allow ready lunch boxes with food to be easily transported from the train-kitchen to the final consumer. The construction and operation of the train are fully financed by The Howard G. Buffett Foundation. Only the latest news, only the facts, only the truth. Follow Ukrainska Pravda on Twitter! The Food Train has passed its test run. "Shortly, a joint team of Ukrzaliznytsia and Buffett's fund will go on permanent duty in the frontline regions of Ukraine where the situation is the most critical and heat and food are vital for survival," Yevhen Liashchenko, head of the Ukrzaliznytsia management, stated. Reportedly, the train consists of six carriages besides the locomotive:
"""
]

In [7]:
# real breaking news:
summary_0 = '20 cars of a Norfolk Southern cargo train derailed near Springfield on Saturday evening. Unlike the 3 February derailment in East Palestine, Ohio, a company spokesperson said there were no hazardous materials aboard the train. The latest train, which did not have passengers, derailed around 5pm on Saturday by state route 41.'

# lawsuit
summary_1 = "Rick Feezle has had a raspy voice and chest pain since the derailment. He's part of a group of people who have filed a class action suit against Norfolk Southern. The Feb. 3 incident resulted in a fire and chemical spill, forcing residents within a roughly 1-mile radius."

# lawsuit + economic/busines impact
summary_2 = "Norfolk Southern recorded another $416 million charge related to the East Palestine derailment. The company faces a number of class-action lawsuits as well as a suit filed by Ohio authorities. The derailment near the Ohio-Pennsylvania border prompted a national conversation about railroad safety."

# economic/business impact
summary_3 = "Norfolk Southern has paid Clark County more than $100,000 in expenses related to one of the company's trains derailing in March in Springfield Twp. Sen. Sherrod Brown (D-OH) sent a letter to Norfolk Southern CEO Alan Shaw last week demanding the company make payments. On March 4 at around 4:45 p.m., 28 train cars containing non-hazardous materials of 212 train cars derailed."

# leisure/general news
summary_4 = 'Ukrzaliznytsia and American billionaire Howard Buffett have created the Food Train. It is able to make up to 70,000 meal portions during the week in an autonomous regime without power. The Food Train can work autonomously for 5-7 days, and its equipment allows for cooking full meals.'

# insurance/business impact
summary_5 = "The Feb. 3 derailment and chemical spill involving a freight train in Ohio is likely to trigger claims on a widening web of property and liability insurance policies, experts say. The incident will have significant risk management and safety implications, and railroads and related businesses can expect greater scrutiny by insurers in the months ahead, they say."

#
summary_6 = "The incident occurred shortly after 7am on Wednesday. The loaded train was travelling from Darfield to Christchurch. The rail line is currently closed so this morning's TranzAlpine service from Christchurch to Greymouth has been replaced by buses. It comes after a train collided with a stolen car on Lincoln Rd in Addington on Tuesday morning. Police are making inquiries into the early morning collision between the stolen Toyota Prius and the train passing through the crossing. It is understood a youth was driving through the level crossing when the car was struck, causing severe damage."


summaries = [summary_0, summary_1, summary_2, summary_3, summary_4, summary_5, summary_6]

In [8]:
# defining premise and hypothesis:
premise = summaries[6]

In [9]:
import torch
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [19]:
hypotheses = [f'This example is {label}.' for label in candidate_labels]
tokens = []
preds = []
device = get_default_device()
print(device)

for hypothesis in hypotheses:
    # tokenizing the input, leave max_length to none to use the max length allowed by the model:
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation=True, max_length=None)
    tokens.append(x)
    # Model Execution: The tokenized input is passed through the model to obtain the logits for each NLI class 
    # the order of the logits: [contradiction, neutral, entailment].
    pred = nli_model(x.to(device))
    preds.append(pred)
    logits = pred.logits # equivalent to pred[0]
    # we are interested in the 2 extremes: contratiction and entail
    entail_contradiction_logits = logits[:,[0,2]]
    print(entail_contradiction_logits)

    # we can now re-normalize these 2 logits with softmax
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:,1]

    print(hypothesis)
    print("prob_label_is_true vs false: " + str(prob_label_is_true.detach().numpy()[0]))
    print("Full softmax logits [contradiction, neutral, entailment]: "+ str(logits.softmax(dim=1).detach().numpy()[0]))
    print("="*30)

cpu
tensor([[-2.4088,  0.4274]], grad_fn=<IndexBackward0>)
This example is very recent breaking news on major railway transportation disruption and closure, bad news.
prob_label_is_true vs false: 0.9446019
Full softmax logits [contradiction, neutral, entailment]: [0.0122495  0.77888244 0.20886807]
tensor([[-0.4748,  0.4918]], grad_fn=<IndexBackward0>)
This example is later reports of past transportation disruption event, bad news.
prob_label_is_true vs false: 0.72443867
Full softmax logits [contradiction, neutral, entailment]: [0.19975917 0.27508274 0.5251581 ]
tensor([[-0.1375, -0.5996]], grad_fn=<IndexBackward0>)
This example is lawsuits, legal or insurance impact of past event, bad news.
prob_label_is_true vs false: 0.38648447
Full softmax logits [contradiction, neutral, entailment]: [0.24247749 0.60477364 0.15274887]
tensor([[-0.0134, -0.6744]], grad_fn=<IndexBackward0>)
This example is general social, business, economic reports, studies and trends.
prob_label_is_true vs false: 0.3

In [25]:
tokens[0]

tensor([[    0,   133,  1160,  2756,  3691,    71,   262,   424,    15,   307,
             4,    20,  7973,  2341,    21,  7290,    31,  5355,  1399,     7,
          4845, 23420,     4,    20,  5569,   516,    16,   855,  1367,    98,
            42,   662,    18,  2393, 14799,  7083, 25638,   544,    31,  4845,
         23420,     7, 10558, 13848,    34,    57,  4209,    30,  8159,     4,
            85,   606,    71,    10,  2341, 15268,    19,    10,  3579,   512,
            15,  5839,  9350,    11,  4287,  6598,    15,   294,   662,     4,
           522,    32,   442, 15099,    88,     5,   419,   662,  7329,   227,
             5,  3579,  7261,  9522,   687,     8,     5,  2341,  3133,   149,
             5,  6724,     4,    85,    16,  6238,    10,  2719,    21,  1428,
           149,     5,   672,  6724,    77,     5,   512,    21,  2322,     6,
          3735,  3814,  1880,     4,     2,     2,   713,  1246,    16,   182,
           485,  3433,   340,    15,   538, 10780,  

In [14]:
result_dict = loaded_classifier_2(premise, candidate_labels)
print(result_dict['sequence'])
pd.DataFrame(data={'labels': result_dict['labels'],
             'scores': result_dict['scores']})

The incident occurred shortly after 7am on Wednesday. The loaded train was travelling from Darfield to Christchurch. The rail line is currently closed so this morning's TranzAlpine service from Christchurch to Greymouth has been replaced by buses. It comes after a train collided with a stolen car on Lincoln Rd in Addington on Tuesday morning. Police are making inquiries into the early morning collision between the stolen Toyota Prius and the train passing through the crossing. It is understood a youth was driving through the level crossing when the car was struck, causing severe damage.


,labels,scores
0,later reports of past transportation disruptio...,0.359578
1,very recent breaking news on major railway tra...,0.337181
2,"lawsuits, legal or insurance impact of past ev...",0.120729
3,"general social, business, economic reports, st...",0.112034
4,leisure news,0.070477


In [22]:
predictions = result_dict
data = {predictions['labels'][i]: predictions['scores'][i] for i in range(len(predictions['labels']))}
print(data)

# Sort the dictionary items by values in descending order
sorted_items = sorted(data.items(), key=lambda item: item[1], reverse=True)
print(sorted_items)
print()
# Get the keys for the highest and second-highest values
highest_key = sorted_items[0][0]
highest_value = sorted_items[0][1]
second_highest_key = sorted_items[1][0]
second_highest_value = sorted_items[1][1]
print(highest_key, highest_value)
print(second_highest_key, second_highest_value)
    # highest_keys[key] = second_highest_key

{'later reports of past transportation disruption event, bad news': 0.359578400850296, 'very recent breaking news on major railway transportation disruption and closure, bad news': 0.33718135952949524, 'lawsuits, legal or insurance impact of past event, bad news': 0.1207287386059761, 'general social, business, economic reports, studies and trends': 0.11203436553478241, 'leisure news': 0.07047703862190247}
[('later reports of past transportation disruption event, bad news', 0.359578400850296), ('very recent breaking news on major railway transportation disruption and closure, bad news', 0.33718135952949524), ('lawsuits, legal or insurance impact of past event, bad news', 0.1207287386059761), ('general social, business, economic reports, studies and trends', 0.11203436553478241), ('leisure news', 0.07047703862190247)]

later reports of past transportation disruption event, bad news 0.359578400850296
very recent breaking news on major railway transportation disruption and closure, bad new

In [23]:
sample_df = pd.read_csv('./sample_summary_with_zero_shot_railway.csv')

In [29]:
import ast
# Convert the strings back to dictionaries using ast.literal_eval
sample_df['ZERO_SHOT_PREDICTION'] = sample_df['ZERO_SHOT_PREDICTION'].apply(ast.literal_eval)

In [ ]:
sample_df['ZERO_SHOT_PREDICTION']

In [38]:
def get_top_n_label_and_score(prediction_dict, top_n):
    try:
        # using the 1st model only:
        first_key = list(prediction_dict.keys())[0]
        data = prediction_dict[first_key]
        # Sort the dictionary items by values in descending order
        sorted_items = sorted(data.items(), key=lambda item: item[1], reverse=True)
        # Get the key and value for top_n value
        top_key = sorted_items[top_n-1][0]
        # top_value = sorted_items[top_n-1][1]
        return top_key
    except:
        return ""


In [39]:
sample_df['TOP_PREDICTION'] = sample_df.apply(lambda row: get_top_n_label_and_score(row['ZERO_SHOT_PREDICTION'],1), axis=1)
sample_df['SECOND_PREDICTION'] = sample_df.apply(lambda row: get_top_n_label_and_score(row['ZERO_SHOT_PREDICTION'],2), axis=1)

In [40]:
sample_df.to_csv('sample_summary_with_zero_shot_railway_03.csv')